In [1]:
import sys
sys.path.append("/home/ubuntu/ins-plt/DS-Inference-Scripts/rl_scripts/retrain/")

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def tranpose(data, index_col, new_index_col):
    data = data.set_index(index_col).T.reset_index().rename(columns={'index': new_index_col})
    data.index.name = None
    data.columns.name = None
    return data

In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp
import traceback
from numpy import matlib as ml
from tqdm import tqdm
from eq_common_utils.utils.config.s3_config import s3_config

s3_helper = s3_config()

bucket = 'micro-ops-output'
prefix = 'test/portfolio-rebalance/hsbc_te300/data/'

/tmp
######################## S3Connection ######################


In [ ]:
benchmark = s3_helper.read_as_dataframe(bucket, prefix+"benchmark.csv")
caps = s3_helper.read_as_dataframe(bucket, prefix + "caps.csv")
closeprices = s3_helper.read_as_dataframe(bucket, prefix + "closeprices.csv")
ers = s3_helper.read_as_dataframe(bucket, prefix + "ers.csv")
sector_weights = s3_helper.read_as_dataframe(bucket, prefix + "sector_weights.csv")
region_weights = s3_helper.read_as_dataframe(bucket, prefix + "region_weights.csv")
selection = s3_helper.read_as_dataframe(bucket, prefix + 'selection.csv')
market_caps = s3_helper.read_as_dataframe(bucket, prefix +"market_caps.csv")

In [6]:
closeprices = closeprices.ffill().bfill().fillna(1.0)

isin_list = sector_weights['isin'].tolist()
common_cols = ['date'] + isin_list
ers = ers[common_cols]
closeprices = closeprices[common_cols]
benchmark = benchmark[common_cols]
caps = caps[common_cols]

In [7]:
def transpose(data_file, index_col, new_index_col):
    data_file = data_file.set_index(index_col).T.reset_index().rename(columns={"index": new_index_col})
    data_file.index.name = None
    data_file.columns.name = None
    return data_file

ers_t = transpose(ers, 'date', 'isin')

In [8]:
def sector_constraints(variable, dataframe):
    constraints = []
    if 'sector_min' in dataframe.columns:
        for sector_name in dataframe['sector_name'].unique():
            mask = dataframe['sector_name'] == sector_name
            if len(dataframe.loc[mask]):
                sector_min = dataframe.loc[mask, 'sector_min'].iloc[0]
                sector_max = dataframe.loc[mask, 'sector_max'].iloc[0]
                sector_w = variable[mask.values]
                constraints.append(cp.sum(sector_w) >= sector_min)
                constraints.append(cp.sum(sector_w) <= sector_max)
            else:
                sector_min = 0
                sector_max = 0
    return constraints

def region_constraints(variable, dataframe):
    constraints = []
    if 'region_max' in dataframe.columns:
        for region_name in dataframe['region_name'].unique():
            mask = dataframe['region_name'] == region_name
            if len(dataframe.loc[mask]):
                region_min = dataframe.loc[mask, 'region_min'].iloc[0]
                region_max = dataframe.loc[mask, 'region_max'].iloc[0]
                region_w = variable[mask.values]
                constraints.append(cp.sum(region_w) >= region_min)
                constraints.append(cp.sum(region_w) <= region_max)
            else:
                region_min = 0
                region_max = 0
    return constraints

In [9]:
daily_returns = closeprices.set_index('date').ffill().bfill().fillna(1).pct_change().dropna()
monthly_returns = closeprices.set_index('date').ffill().bfill().fillna(1).pct_change(periods=22).dropna()

In [10]:
window = 512
mult_values = 10**(np.linspace(-4,4,10))

In [11]:
def nonlinear_shrink_cov(data):
    
    EPS = 1e-8
    n_obs, p_assets = data.shape
    assert n_obs >= 12, "sample size must be >= 12"
    data = data - data.mean(axis=0)
    n = n_obs - 1
    sample_cov = (data.T @ data) / n
    lam, u = np.linalg.eigh(sample_cov)
    if p_assets > n:
        lam = lam[p_assets - n:]
        p_eff = p_assets
    else:
        tol = 1e-12 * lam.max()
        keep = lam > tol
        lam = lam[keep]
        u = u[:, keep]
        p_eff = lam.size
    h = n ** (-1 / 3)
    L = ml.repmat(lam.T, min(p_eff, n), 1).T

    H = h * L.T
    x = (L - L.T) / H

    ftilde = (3.0 / (4.0 * np.sqrt(5.0))) * np.mean(
        np.maximum(1.0 - x**2 / 5.0, 0.0) / H,
        axis=1,
    )

    Hftemp = (
        (-3.0 / (10.0 * np.pi)) * x
        + (3.0 / (4.0 * np.sqrt(5.0) * np.pi))
        * (1.0 - x**2 / 5.0)
        * np.log(np.abs((np.sqrt(5.0) - x) / (np.sqrt(5.0) + x)))
    )

    mask = np.abs(x) == np.sqrt(5.0)
    Hftemp[mask] = (-3.0 / (10.0 * np.pi)) * x[mask]

    Hftilde = np.mean(Hftemp / H, axis=1)
    if p_assets <= n:
        dtilde = lam / (
            (np.pi * (p_eff / n) * lam * ftilde) ** 2
            + (1.0 - (p_eff / n) - np.pi * (p_eff / n) * lam * Hftilde) ** 2
        )
    else:
        Hftilde0 = (
            (1.0 / np.pi)
            * (
                3.0 / (10.0 * h**2)
                + 3.0 / (4.0 * np.sqrt(5.0) * h)
                * (1.0 - 1.0 / (5.0 * h**2))
                * np.log((1.0 + np.sqrt(5.0) * h) / (1.0 - np.sqrt(5.0) * h))
            ) 
            * np.mean(1.0 / lam)
        )

        dtilde0 = 1.0 / (np.pi * (p_eff - n) / n * Hftilde0)
        dtilde1 = lam / (np.pi**2 * lam**2 * (ftilde**2 + Hftilde**2))
        dtilde = np.concatenate([dtilde0 * np.ones(p_eff - n), dtilde1])
    sigmatilde = u @ np.diag(dtilde) @ u.T
    return sigmatilde


In [12]:
import numpy as np

def get_risk(w, df, cov):

    selected = df['selected'].astype(bool).values
    non_selected = ~selected

    b = df['benchmark'].values
    b_p = b[selected]
    b_n = b[non_selected]

    # ---- NumPy slicing FIRST ----
    Σ_pp = cov[np.ix_(selected, selected)]
    Σ_pn = cov[np.ix_(selected, non_selected)]
    Σ_nn = cov[np.ix_(non_selected, non_selected)]

    # ---- Symmetrize square blocks ----
    Σ_pp = 0.5 * (Σ_pp + Σ_pp.T)
    Σ_nn = 0.5 * (Σ_nn + Σ_nn.T)

    # ---- Wrap PSD matrices ----
    Σ_pp = cp.psd_wrap(Σ_pp)
    Σ_nn = cp.psd_wrap(Σ_nn)

    w_p = w[selected]
    diff_p = w_p - b_p

    return (
        cp.quad_form(diff_p, Σ_pp)
        + 2 * cp.sum(cp.multiply(diff_p, Σ_pn @ (-b_n)))
        + b_n.T @ Σ_nn @ b_n
    )

In [41]:
def max_er(df, cov, cov_m, sector_w, region_w, verbose):

    er = df['er'].values
    caps = df['caps'].values
    b = df['benchmark'].values
    min_ = df['min'].values

    caps = np.maximum(caps, min_)
    
    n = len(er)
    w = cp.Variable(n, nonneg=True)
    
    objective_returns = w.T @ er
    risk = get_risk(w, df, cov)
    constraints = [
        w <= caps,
        w >= min_,
        cp.sum(w) == 1,
        risk <= risk_limit
    ]
    constraints += sector_constraints(w, sector_w)
    constraints += region_constraints(w, region_w)

    objective = cp.Maximize(objective_returns)
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.SCS, verbose=verbose)

    if w.value is not None:
        return w.value
    else:
        return None  

In [42]:
te_threshold = 0.03
risk_limit = np.power(te_threshold, 2) / 252

In [43]:
rebalance_dates = ers_t.columns[1:].tolist()

In [35]:
def region_maximum(row, date):
    if row['region_name'] == 'OTHERS':
        return 1
    else:
        return row[date]

In [46]:
import math
def select_isins(df, col, n_isins=500):

    selection = []
    sector_weights = df.groupby('sector_name')['benchmark'].sum().reset_index()
    sector_weights = sector_weights[sector_weights['sector_name'] != 'NA']
    sector_weights['benchmark'] = sector_weights['benchmark']/sector_weights['benchmark'].sum()
    for iteration, sector_info in sector_weights.iterrows():
        sector_name = sector_info['sector_name']
        sector_wts = sector_info['benchmark']

        n_sector_isins = int(math.ceil(n_isins * sector_wts))
        sector_df = df[df['sector_name'] == sector_name].reset_index(drop=True)
        sector_df = sector_df[sector_df['selected'] > 0].sort_values(by=col, ascending=False).reset_index(drop=True).iloc[:n_sector_isins]
        selection.append(sector_df)
    
    return pd.concat(selection).reset_index(drop=True)['isin'].tolist()

In [47]:
verbose = False

def solve_one_date(date):
    try:
        s3_helper = s3_config()
        df = benchmark.set_index('date').loc[date].reset_index().rename(columns={'index': 'isin', date: 'benchmark'})
        df = df[df['benchmark'] > 0].reset_index(drop=True)
        isins_ordered = df['isin'].tolist()
        caps_window = caps.set_index('date').loc[date, isins_ordered].values
        er_window = ers.set_index('date').loc[date, isins_ordered].values
        caps_map = dict(zip(isins_ordered, caps_window))
        er_map = dict(zip(isins_ordered, er_window))
        df['caps'] = df['isin'].map(caps_map).fillna(0.0)
        df['er'] = df['isin'].map(er_map)
        df['selected'] = selection.set_index('date')[isins_ordered].loc[date].values

        sector_window = sector_weights.set_index('isin').loc[isins_ordered][['sector_name', date]].copy()
        sector_window.fillna({"sector_name" : "NA"}, inplace=True)
        region_window = region_weights.set_index('isin').loc[isins_ordered][['region_name', date]].copy()
        region_window['region_min'] = 0.0
        region_window['region_max'] = region_window[['region_name', date]].apply(lambda row: region_maximum(row, date), axis=1)

        sector_window['sector_min'] = (sector_window[date] * 0.5)
        sector_window['sector_max'] = sector_window[date] * 1.5

        df['sector_name'] = sector_window['sector_name'].values
        df['region_name'] = region_window['region_name'].values

        daily_window = daily_returns[isins_ordered].loc[:date].tail(window)
        monthly_window = monthly_returns[isins_ordered].loc[:date].tail(window)

        cov_m = nonlinear_shrink_cov(monthly_window)
        cov_d = nonlinear_shrink_cov(daily_window)

        if df['selected'].sum() > 500:
            df_selection = select_isins(df, 'benchmark')
            df['selected'] = df['isin'].isin(df_selection).astype(int)
            df['min'] = 0
        df['caps'] = df['caps'] * df['selected']

        wts = max_er(df, cov_d, cov_m, sector_window, region_window, verbose)
        if wts is None:
            sector_window['sector_min'] = (sector_window[date] * 0)
            sector_window['sector_max'] = sector_window[date] * 1.5
            wts = max_er(df, cov_d, cov_m, sector_window, region_window, verbose)
        
        df['Wts'] = wts
        if df['selected'].sum() > 250:
            df_selection = select_isins(df, 'Wts', n_isins=250)
            df['selected'] = df['isin'].isin(df_selection).astype(int)
        df['min'] = 0.001
        df['min'] = df['min'] * df['selected']
        df['caps'] = df['caps'] * df['selected']

        wts = max_er(df, cov_d, cov_m, sector_window, region_window, verbose)
        df['Wts-II'] = wts
        path = prefix + f'{date}/Weights.csv'
        s3_helper.write_advanced_as_df(df, bucket, path)
    except KeyboardInterrupt:
        pass
    except Exception as e:
        print(f"date failed because of {traceback.format_exc()}")

In [48]:
isin_list = benchmark.columns.tolist()[1:]
rebalance_dates = ers['date'].tolist()

In [49]:
prefix = 'test/portfolio-rebalance/hsbc_te300/weights/max_er_3/'

In [ ]:
from joblib import Parallel, delayed
from tqdm import tqdm

weights_df = pd.DataFrame(
    0.0,
    index=rebalance_dates,
    columns=isin_list
)

results = Parallel(
    n_jobs=-1,         
    backend="loky",     
    verbose=0
)(
    delayed(solve_one_date)(date)
    for date in tqdm(rebalance_dates)
)


/tmp
/tmp
/tmp
/tmp
/tmp
/tmp
/tmp
/tmp
/tmp
/tmp
/tmp
/tmp
/tmp
/tmp
/tmp
/tmp
######################## S3Connection ######################
######################## S3Connection ######################
######################## S3Connection ######################
######################## S3Connection ############################################## S3Connection ######################

######################## S3Connection ######################
######################## S3Connection ######################
######################## S3Connection ######################
######################## S3Connection ######################
######################## S3Connection ######################
######################## S3Connection ######################
######################## S3Connection ######################
######################## S3Connection ######################
######################## S3Connection ######################
######################## S3Connection ######################
#####

isin            object
benchmark      float64
caps           float64
er             float64
selected         int64
sector_name     object
region_name     object
min            float64
Wts            float64
Wts-II         float64
dtype: object
columns ->  isin
0       AN8068571086
1       AT0000606306
2       AT0000652011
3       AT0000720008
4       AT0000741053
            ...     
1640    US98421M1062
1641    US9884981013
1642    US98956P1021
1643    US9897011071
1644    ZAE000255360
Name: isin, Length: 1645, dtype: object
columns ->  benchmark
0       0.003054
1       0.000129
2       0.000452
3       0.000319
4       0.000033
          ...   
1640    0.000519
1641    0.000563
1642    0.000611
1643    0.000338
1644    0.000591
Name: benchmark, Length: 1645, dtype: float64
columns ->  caps
0       0.031151
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
1640    0.008069
1641    0.001336
1642    0.005099
1643    0.000000
1644    0.000245
Name: cap

/home/ubuntu/miniconda3/envs/TTM-EXPECTED-VALUES/lib/python3.11/site-packages/cvxpy/problems/problem.py:1510: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


isin            object
benchmark      float64
caps           float64
er             float64
selected         int64
sector_name     object
region_name     object
min            float64
Wts            float64
Wts-II         float64
dtype: object
columns ->  isin
0       AN8068571086
1       AT0000606306
2       AT0000642806
3       AT0000652011
4       AT0000720008
            ...     
1609    US98421M1062
1610    US9884981013
1611    US98956P1021
1612    US9897011071
1613    ZAE000255360
Name: isin, Length: 1614, dtype: object
columns ->  benchmark
0       0.002915
1       0.000221
2       0.000151
3       0.000541
4       0.000343
          ...   
1609    0.000598
1610    0.000560
1611    0.000749
1612    0.000321
1613    0.000547
Name: benchmark, Length: 1614, dtype: float64
columns ->  caps
0       0.031025
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
1609    0.008603
1610    0.000000
1611    0.005072
1612    0.000000
1613    0.000253
Name: cap

/home/ubuntu/miniconda3/envs/TTM-EXPECTED-VALUES/lib/python3.11/site-packages/cvxpy/problems/problem.py:1510: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


######################## S3Connection ######################
######################## S3Connection ######################
isin            object
benchmark      float64
caps           float64
er             float64
selected         int64
sector_name     object
region_name     object
min            float64
Wts            float64
Wts-II         float64
dtype: object
columns ->  isin
0       AN8068571086
1       AT0000606306
2       AT0000642806
3       AT0000652011
4       AT0000720008
            ...     
1613    US98421M1062
1614    US9884981013
1615    US98956P1021
1616    US9897011071
1617    ZAE000255360
Name: isin, Length: 1618, dtype: object
columns ->  benchmark
0       0.002689
1       0.000240
2       0.000152
3       0.000547
4       0.000375
          ...   
1613    0.000586
1614    0.000596
1615    0.000753
1616    0.000323
1617    0.000573
Name: benchmark, Length: 1618, dtype: float64
columns ->  caps
0       0.038259
1       0.000000
2       0.000000
3       0.000000
4     

/home/ubuntu/miniconda3/envs/TTM-EXPECTED-VALUES/lib/python3.11/site-packages/cvxpy/problems/problem.py:1510: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(



isin            object
benchmark      float64
caps           float64
er             float64
selected         int64
sector_name     object
region_name     object
min            float64
Wts            float64
Wts-II         float64
dtype: object
columns ->  isin
0       AN8068571086
1       AT0000606306
2       AT0000642806
3       AT0000652011
4       AT0000720008
            ...     
1618    US9884981013
1619    US98956P1021
1620    US9897011071
1621    ZAE000156550
1622    ZAE000255360
Name: isin, Length: 1623, dtype: object
columns ->  benchmark
0       0.004132
1       0.000214
2       0.000154
3       0.000506
4       0.000300
          ...   
1618    0.000605
1619    0.000658
1620    0.000247
1621    0.000050
1622    0.000494
Name: benchmark, Length: 1623, dtype: float64
columns ->  caps
0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
1618    0.000000
1619    0.005618
1620    0.000000
1621    0.000000
1622    0.000220
Name: cap

In [ ]:
for date in rebalance_dates:
    if 

'test/portfolio-rebalance/hsbc_te300/weights/max_er_3/'